# Span: Spectral Analysis

#### Span is a set of tools to fit spectra the of binary stars simultaneously.

#### This means that we need synthetic atmosphere models to do the fit. In this example we use [TLUSTY](http://tlusty.oca.eu) models for stars hotter than 15000 K, and ATLAS 9 models (from [Howarth 2011](https://ui.adsabs.harvard.edu/abs/2011MNRAS.413.1515H/abstract)) for temperatures 10000-15000 K, but you can use your favourite models.

#### If you use any of these models in your work (available at minato>models) please use the correct reference:

#### - ATLAS 9 late B-type stars grid: [Howarth 2011](https://ui.adsabs.harvard.edu/abs/2011MNRAS.413.1515H/abstract)
#### - TLUSTY B-type stars grid: [Lanz & Hubeny 2007](https://ui.adsabs.harvard.edu/abs/2007ApJS..169...83L/abstract)
#### - TLUSTY O-type stars grid: [Lanz & Hubeny 2003](https://ui.adsabs.harvard.edu/abs/2003ApJS..146..417L/abstract)

Let's start by creating a simple grid of parameters. Parameters currently accepted by Span are: effective temperature (both stars) surface gravity (both stars), rotational velocity (both stars), the light ratio (defined as the contribution of the secondary star), and the Helium enrichment (primary star).

Here we'll create a grid similar to the one of [Villaseñor+2023](https://ui.adsabs.harvard.edu/abs/2023arXiv230707766V/abstract) but less dense. How dense and broad is your grid will depend on the information you have on your binary. Note that you can do this using `numpy`, `range`, or a simple list `[]`

This grid should run for about 20 minutes, so to make it faster for this example let's fix the rotational velocity of the companion (rotB). It should now be just a couple of minutes.

In [88]:
import numpy as np

teffA = [15, 20, 25, 30]
loggA = [21, 23, 25, 27, 29, 31, 33, 35]
rotA = [40]
micA = [5, 9, 13, 17, 21, 25]
# micA = []
teffB = [15, 20, 25, 30]
loggB = [25, 27, 29, 31, 33, 35]
# rotB = list(range(0, 600, 100))
rotB = [40]
micB = [5]
lrat = np.arange(5, 30, 5)/100
# lrat = [0.05]
he2h = np.linspace(10, 20, 5)/100
# he2h = []
pars_list = [lrat, he2h, teffA, loggA, rotA, micA, teffB, loggB, rotB, micB]

gridlist = [list(l) for l in pars_list]
for par in gridlist:
    print(par)

[0.05, 0.1, 0.15, 0.2, 0.25]
[0.1, 0.125, 0.15, 0.175, 0.2]
[15, 20, 25, 30]
[21, 23, 25, 27, 29, 31, 33, 35]
[40]
[5, 9, 13, 17, 21, 25]
[15, 20, 25, 30]
[25, 27, 29, 31, 33, 35]
[40]
[5]


In [89]:
# changing list of lists to a dictionary
# name of the models will need to be set to match the dictionary keys
# e.g. Teff_logg_Y(He)_logQ_beta_abundance_vmic_vsini

grid = {
    'lr': np.arange(5, 30, 5)/100,
    'TA': [15000, 20000, 25000, 30000],
    'gA': [210, 230, 250, 270, 290, 310, 330, 350],
    'mA': [5, 9, 13, 17, 21, 25],
    'vA': [40],
    'TB': [15000, 20000, 25000, 30000],
    'gB': [250, 270, 290, 310, 330, 350],
    'mB': [5, 9],
    'vB': [40]
}


>
>#### **Note:** the light ratio is defined as flux_secondary / total_flux
>

In the next step we need to specify our observed spectra. In our example we're using the disentangled spectra from [Villaseñor+2023](https://ui.adsabs.harvard.edu/abs/2023arXiv230707766V/abstract).

And also we have to specify what spectral lines we want to be considered in the fit

In [90]:
dsnt_A = './example_spectra/ADIS_lguess_K1K2=0.3_94.0_15.0.txt'
dsnt_B = './example_spectra/BDIS_lguess_K1K2=0.3_94.0_15.0.txt'

select_linesA = [4026, 4102, 4121, 4144, 4267, 4340, 4388, 4471, 4553]
select_linesB = [4026, 4102, 4121, 4144, 4267, 4340, 4388, 4553]

>
>#### **Note:** make sure the models are interpolated to the disentangled spectra wavelength or to a common wavelength grid
>

First we need to interpolate the models to the wavelength grid of the spectra or to a common wavelength grid. In this case we'll use the wavelength of the disentangled spectra

In [131]:
import sys
sys.path.append('/Users/villasenor/science/github/jvillasr/MINATO')
from minato import span as sp
import importlib
importlib.reload(sp)

# Path to de original and interpolated models
path = '/Users/villasenor/science/projects/BBC/VFTS_779/fastwind_models/redprueba_fastwind_jaime/'
outpath = '/Users/villasenor/science/KUL_research/models/fastwind_models/redprueba_fastwind_jaime/models_tutorial/'

# Creating the atmfit object
fit_obj = sp.atmfit(dsnt_A, dsnt_B, grid=grid, lrat0=0.3, modelsA_path=outpath, modelsB_path=outpath, binary=True, crop_nebular=True)
# Get wavelength from the observed spectra
wavA, wavB = fit_obj.get_wave()


In [103]:
# Interpolating the models
fit_obj.interpolate_models(models_path=path, models_extension='txt', wavelength=wavA, output_path=outpath)

One final check, is to make sure that the names of the models contain only the keys and values given in the grid. We're  going to make a ffew adjustments to the name of our models

In [ ]:
import os

def massage_model_name(name):
    # Remove the "maui_" prefix
    name = name.replace("maui_", "")

    # Remove parameters that do not vary
    name = name.replace("He10Q135_C800_N780_O839_Si719_Mg719", "")

    # Insert an underscore before "g"
    # name = name.replace("g", "_g")

    # Remove the "_R85000" suffix
    name = name.replace("_R85000", "")

    # Remove underscores
    name = name.replace("_", "")

    return name

# Get a list of all files in the directory
model_files = os.listdir(outpath)

# Loop over each file
for old_name in model_files:
    # Get the new name
    new_name = massage_model_name(old_name)

    # Get the full paths to the old and new files
    old_path = os.path.join(outpath, old_name)
    new_path = os.path.join(outpath, new_name)

    # Rename the file
    os.rename(old_path, new_path)

We're now ready to use `compute_chi2` to do the actual fit

In [132]:
df_results = fit_obj.compute_chi2(select_linesA, select_linesB)
display(df_results)

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


100%|██████████| 46080/46080 [00:34<00:00, 1328.75it/s]


Computation completed in: 0:00:38.071325 [s] 



,lr,TA,gA,mA,vA,TB,gB,mB,vB,chi2_tot,chi2A,chi2B,chi2r_tot,chi2redA,chi2redB,ndata
0,0.05,15000,210,5,40,15000,250,5,40,82.453610,1.807956,80.645654,0.083792,0.001585,0.082208,2140
1,0.05,15000,210,5,40,15000,250,9,40,82.154997,1.807956,80.347040,0.083488,0.001585,0.081903,2140
2,0.05,15000,210,5,40,20000,250,5,40,89.405148,1.807956,87.597191,0.090878,0.001585,0.089294,2140
3,0.05,15000,210,5,40,20000,250,9,40,88.400464,1.807956,86.592507,0.089854,0.001585,0.088270,2140
4,0.05,15000,210,5,40,20000,270,5,40,86.033134,1.807956,84.225178,0.087441,0.001585,0.085856,2140
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9235,0.25,30000,350,25,40,30000,310,9,40,7.865818,4.464497,3.401321,0.007380,0.003913,0.003467,2140
9236,0.25,30000,350,25,40,30000,330,5,40,6.900874,4.464497,2.436377,0.006396,0.003913,0.002484,2140
9237,0.25,30000,350,25,40,30000,330,9,40,7.198963,4.464497,2.734466,0.006700,0.003913,0.002787,2140
9238,0.25,30000,350,25,40,30000,350,5,40,6.423651,4.464497,1.959154,0.005910,0.003913,0.001997,2140


Now that we have our dataframe with the results, we should save it to work with it later if we want to. Since this can get very heavy, I recommend to use the feather format

In [134]:
df_results.to_feather('atmfit_results_280624_test.feather')

See the tutorial "__Working with atmfit results__" to see how to get the best-fitting parameters